<a href="https://colab.research.google.com/github/naveenkumarse/Amaron_ecom/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [3]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [4]:
!pip install -qqq langchain llama-index llama-hub openai accelerate==0.21.0 bitsandbytes==0.40.2 transformers sentence_transformers InstructorEmbedding chromadb pypdf pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6

In [5]:
!pip install pypdf pdf2image

In [6]:
!pip install -qqq langchain llama-index llama-hub openai accelerate bitsandbytes transformers sentence_transformers InstructorEmbedding chromadb pypdf pdf2image

In [7]:
import torch
import logging
import sys

# llama_index
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage, ServiceContext
from langchain.vectorstores import Chroma

from transformers import AutoTokenizer, TextStreamer, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer


# llama_index
from llama_index.response.notebook_utils import display_response
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor
from llama_index.response_synthesizers import get_response_synthesizer
from llama_index import SummaryIndex
from llama_index.text_splitter import SentenceSplitter
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.response.pprint_utils import pprint_response
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM

# chromadb
import chromadb

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# ChatBot using LlamaIndex

# Data Connectors

In [8]:
pip install docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=7fafcc04537e7d628d59054ed33497f0cb7e57911455fa700f9a7bcbcfe02853
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [9]:
# Earning reports of 3 banks
!ls './sample_data/pdfs'

KEC_02.docx


In [10]:
# Load data
documents = SimpleDirectoryReader('./sample_data/pdfs').load_data()
len(documents)

1

# Models

## LLM

In [11]:
from google.colab import userdata

# huggingface api token for downloading llama2
hf_token = userdata.get('hf')

In [12]:
!pip install bitsandbytes

In [13]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    query_wrapper_prompt=PromptTemplate("<s> [INST] {query_str} [/INST] "),
    context_window=3900,
    model_kwargs={"token": hf_token, "quantization_config": quantization_config},
    tokenizer_kwargs={"token": hf_token},
    device_map="auto",
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Embedding Model

In [15]:
embed_model = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


# Defining the Service Context

In [16]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20) # using the default chunk_# values as they work just fine

# set context window
context_window = 4096
# set number of output tokens
num_output = 256

service_context = ServiceContext.from_defaults(llm=llm,
                                               embed_model=embed_model,
                                               text_splitter=text_splitter,
                                              #  context_window=context_window,
                                              #  num_output=num_output,
                                               )

# Index Setup

In [17]:
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [18]:
summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

# Storing

In [ ]:
# # # To make ephemeral client that is a short lasting client or an in-memory client
# # db = chromadb.EphemeralClient()

# # initialize client, setting path to save data
# db = chromadb.PersistentClient(path="./chroma_db")

# # create collection
# chroma_collection = db.get_or_create_collection("bank_earnings_database")

# # assign chroma as the vector_store to the context
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

# # create your index
# vector_index = VectorStoreIndex.from_documents(
#     documents,
#     storage_context=storage_context,
#     service_context=service_context
# )

# Querying

### Compact (default)

In [19]:
# create a query engine and query

query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("What is the principal of Kongu Engineering College?")

pprint_response(response, show_source=True)

Final Response: Based on the context information provided, the
principal of Kongu Engineering College is Dr. K. Kannan.
______________________________________________________________________
Source Node 1/2
Node ID: 6d8e05a0-9ca7-4ea8-b51b-d846c60ee03d
Similarity: 0.9058360282246399
Text: theses, contribute to making Kongu Engineering College's library
a robust and comprehensive center for academic excellence.  At Kongu
engineering college, there are around 3 ladies hostels, 7 boys
hostels, 1 Guest house, 1 Alumni Guest house and a separate Staff
Quarters.  There are around 3 ladies hostels in kongu engineering
college namely Kav...
______________________________________________________________________
Source Node 2/2
Node ID: 46600d20-437f-46f4-b4b8-05634cdf068e
Similarity: 0.9045399780396048
Text: This accreditation is valid up to March 28, 2027, affirming the
institution's commitment to maintaining high academic standards and
quality education. The A++ Grade reflects Kongu Engineeri

In [20]:
response.source_nodes[0]

NodeWithScore(node=TextNode(id_='6d8e05a0-9ca7-4ea8-b51b-d846c60ee03d', embedding=None, metadata={'file_name': 'KEC_02.docx', 'file_path': 'sample_data/pdfs/KEC_02.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 46465, 'creation_date': '2024-01-20', 'last_modified_date': '2024-01-20', 'last_accessed_date': '2024-01-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d5d19598-df69-47de-bec6-6c61f1a2ced7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_name': 'KEC_02.docx', 'file_path': 'sample_data/pdfs/KEC_02.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 46465, 'creation_date': 

In [21]:
response.source_nodes[1]

NodeWithScore(node=TextNode(id_='46600d20-437f-46f4-b4b8-05634cdf068e', embedding=None, metadata={'file_name': 'KEC_02.docx', 'file_path': 'sample_data/pdfs/KEC_02.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 46465, 'creation_date': '2024-01-20', 'last_modified_date': '2024-01-20', 'last_accessed_date': '2024-01-20'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d5d19598-df69-47de-bec6-6c61f1a2ced7', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_name': 'KEC_02.docx', 'file_path': 'sample_data/pdfs/KEC_02.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 46465, 'creation_date': 

In [22]:
response = query_engine.query("What are the regular working hours in Kongu Engineering College?")

pprint_response(response, show_source=True)

Final Response: Based on the provided context information, the regular
working hours of Kongu Engineering College are as follows:  Monday to
Friday: 9:00 AM - 5:00 PM Saturday: 9:00 AM - 1:00 PM  Note: These
working hours may be subject to change, and it is always best to
confirm with the college administration or visit their website for the
most up-to-date information.
______________________________________________________________________
Source Node 1/2
Node ID: 6d8e05a0-9ca7-4ea8-b51b-d846c60ee03d
Similarity: 0.9023041189694319
Text: theses, contribute to making Kongu Engineering College's library
a robust and comprehensive center for academic excellence.  At Kongu
engineering college, there are around 3 ladies hostels, 7 boys
hostels, 1 Guest house, 1 Alumni Guest house and a separate Staff
Quarters.  There are around 3 ladies hostels in kongu engineering
college namely Kav...
______________________________________________________________________
Source Node 2/2
Node ID: 46600d20-4

In [23]:
response = query_engine.query("What is KVIT Trust?")

display_response(response)

**`Final Response:`** Based on the provided context information, KVIT Trust is the Kongu Vellalar Institute of Technology Trust, which was established in 1983 by a group of philanthropists from the Kongu Vellalar Gounder community in Erode District, Tamil Nadu, India. The trust's mission is to promote technical education and cultural activities, as well as to encourage literary and scientific knowledge. The trust is responsible for the establishment and operation of Kongu Engineering College, which offers various undergraduate and postgraduate programs in engineering and technology, as well as other programs such as Naturopathy, Master of Business Administration (MBA), Master of Computer Applications (MCA), and Bachelor of Science (B.Sc). The trust's vision is to be a center of excellence for the development and dissemination of knowledge in applied science, technology, engineering, and management for the nation and beyond.

In [24]:
response = query_engine.query("How does the HDFC Life Insurance performance looks like in Q2FY24?")

display_response(response)

**`Final Response:`** Based on the context information provided, there is no mention of HDFC Life Insurance or any performance data related to it. Therefore, it is not possible to provide an answer to the query about HDFC Life Insurance's performance in Q2FY24.

In [25]:
response = query_engine.query("When was the Computer centre opened in the college?")

display_response(response)

**`Final Response:`** Based on the context information provided, the Computer Centre was established in the college in 1996.

### Refine

In [ ]:
# query_engine = vector_index.as_query_engine(response_mode="refine")

# response = query_engine.query("Can you compare the Gross NPA and Net NPA of HDFC, ICICI and Axis Bank in the latest quarter?")

# display_response(response)

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


**`Final Response:`** Based on the context information provided, it seems that the article is discussing Kongu Engineering College and its various initiatives, including the establishment of a Center of Excellence (CoE). There is no mention of any banks or financial institutions, including HDFC, ICICI, and Axis Bank, in the context information provided. Therefore, I cannot compare the Gross NPA and Net NPA of these banks in the latest quarter.

It's important to note that Gross NPA and Net NPA are financial terms that are commonly used in the banking and finance industry, but they are not directly related to the context of Kongu Engineering College or its initiatives. Therefore, I cannot provide an answer to the query based on the information provided in the context.

In [ ]:
# query_engine_refined = vector_index.as_query_engine(response_mode="refine")

# response = query_engine_refined.query("List some of the acts and regulations related to coal mining in India? give me all the laws")

# display_response(response)

**`Final Response:`** Thank you for providing more context! Based on the additional information provided, here is a refined answer to the query:

The acts and regulations related to coal mining in India are as follows:

1. The Coal Mines Act, 1952 - This act provides the legal framework for the mining of coal in India. It regulates the ownership, operation, and management of coal mines, and sets out the rights and obligations of coal mine owners, operators, and workers.
2. The Coal Bearing Areas (Acquisition of Land) Act, 1957 - This act provides for the acquisition of land for coal mining purposes. It authorizes the Central Government to acquire land for coal mining and to pay compensation to the landowners.
3. The Coal Mines (Nationalization) Act, 1973 - This act nationalized the coal mines in India, with the aim of increasing the supply of coal for the public sector and controlling the price of coal.
4. The Coal Mines (Conservation and Development) Act, 1974 - This act provides for the conservation and development of coal mines

In [ ]:
# query_engine_refined = vector_index.as_query_engine(response_mode="refine")

# response = query_engine_refined.query("What steps HDFC Bank is taking towards sustainablity?")

# display_response(response)

**`Final Response:`** Based on the new context provided, here is a refined answer to the original query:

HDFC Bank is taking steps towards sustainability by:

1. Investing in green branches: HDFC Bank is committed to having all new branches certified as green, which suggests that they are prioritizing sustainability in their expansion efforts.
2. Financing renewable energy: HDFC Bank has financed 6,110 MW of renewable energy capacity as of March 31, 2023, which demonstrates their commitment to reducing their carbon footprint.
3. Supporting businesses and communities: HDFC Bank is focused on supporting businesses and enabling smart banking, as well as empowering communities through their CSR initiatives. They have cumulatively impacted over 99.3 million beneficiaries through their CSR efforts.
4. Leading responsibly: HDFC Bank is committed to leading responsibly, which includes nurturing, caring, and collaborating with stakeholders. They have also established a 2-tiered governance structure for diversity, equity, and inclusion at

### Tree Summarize

A Summary Index is a simpler form of Index best suited to queries where, as the name suggests, you are trying to generate a summary of the text in your Documents. It simply stores all of the Documents and returns all of them to your query engine.

A summarization query requires the LLM to iterate through many if not most documents in order to synthesize an answer. For instance, a summarization query could look like one of the following:
•	“What is a summary of this collection of text?”
•	“Give me a summary of person X’s experience with the company.”
In general, a summary index would be suited for this use case. A summary index by default goes through all the data.
Empirically, setting response_mode="tree_summarize" also leads to better summarization results.


In [ ]:
%%time
query_engine_summarized = vector_index.as_query_engine(response_mode="tree_summarize")

response = query_engine_summarized.query("Can you summarize the earnings report of Axis Bank?”")

display_response(response)

**`Final Response:`** Based on the information provided in the given context, here is a summary of the earnings report of Axis Bank:

* Axis Bank reported a net profit of ₹118 billion in Q2FY24, a growth of 26% YoY.
* The bank's total income increased by 14% YoY to ₹324 billion in Q2FY24.
* The bank's net interest income increased by 17% YoY to ₹144 billion in Q2FY24.
* The bank's provisions for bad loans increased by 31% YoY to ₹20 billion in Q2FY24.
* The bank's capital adequacy ratio (CAR) was 14.56%, while its common equity tier 1 (CET 1) ratio was 118 billion.
* The bank's credit card business saw a growth of 2% YoY, while its deposit growth was 16% YoY.
* The bank's advances grew by 114% Yo

CPU times: user 21.3 s, sys: 309 ms, total: 21.6 s
Wall time: 23.9 s


# Customize Your Retrieval Query

In [ ]:
# configure retriever
retriever = VectorIndexRetriever(
    index=vector_index,
    similarity_top_k=6,
)

# configure node postprocessors
s_processor = SimilarityPostprocessor(similarity_cutoff=0.79)
k_processor = KeywordNodePostprocessor(
    exclude_keywords=["environmental"]
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(service_context=service_context)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[s_processor, k_processor],
    response_synthesizer=response_synthesizer
)

In [ ]:
response = query_engine.query("Tell about the mineral conservation and development rules 2017?")

display_response(response)

Number of requested results 6 is greater than number of elements in index 1, updating n_results = 1


**`Final Response:`** Empty Response

Use MMR **maximum marginal relevance**

Instead of ranking vectors purely by similarity, adds diversity to the documents by penalizing documents similar to ones that have already been found based on MMR . A lower mmr_treshold increases diversity.

In [ ]:
# query_engine = vector_index.as_query_engine(
#     vector_store_query_mode="mmr", vector_store_kwargs={"mmr_threshold": 0.2}
# )
# response_mmr = query_engine.query("What steps HDFC Bank is taking towards sustainablity?")

## Automated Metadata Extraction for Better Document Indexing & Retrieval

In many cases, especially with long documents, a chunk of text may lack the context necessary to disambiguate the chunk from other similar chunks of text. One method of addressing this is manually labelling each chunk in our dataset or knowledge base. However, this can be labour intensive and time consuming for a large number or continually updated set of documents.

To combat this, we use LLMs to extract certain contextual information relevant to the document to better help the retrieval and language models disambiguate similar-looking passages.

We do this through Metadata Extractor modules.

You can use LLMs to automate metadata extraction with our Metadata Extractor modules.


1. **SummaryExtractor** - automatically extracts a summary over a set of Nodes

2. **QuestionsAnsweredExtractor** - extracts a set of questions that each Node can answer

3. **TitleExtractor** - extracts a title over the context of each Node

4. **EntityExtractor** - extracts entities (i.e. names of places, people, things) mentioned in the content of each Node

Then you can chain the Metadata Extractors with the node parser

In [ ]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)

from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import MetadataMode

In [ ]:
node_parser = TokenTextSplitter(
    separator=" ", chunk_size=256, chunk_overlap=128
)


extractors_1 = [
    QuestionsAnsweredExtractor(
        questions=3, llm=llm, metadata_mode=MetadataMode.EMBED
    ),
]

extractors_2 = [
    SummaryExtractor(summaries=["prev", "self", "next"], llm=llm),
    QuestionsAnsweredExtractor(
        questions=3, llm=llm, metadata_mode=MetadataMode.EMBED
    ),
]

In [ ]:
extractors_2 = [
    SummaryExtractor(summaries=["prev", "self", "next"], llm=llm),
    QuestionsAnsweredExtractor(
        questions=3, llm=llm, metadata_mode=MetadataMode.EMBED
    ),
]

In [ ]:
orig_nodes = node_parser.get_nodes_from_documents(documents)

In [ ]:
# take just the first 8 nodes for testing
nodes = orig_nodes[10:14]

In [ ]:
print(nodes[3].get_content(metadata_mode="all"))

IndexError: list index out of range

In [ ]:
# Run metadata extractors
from llama_index.ingestion import IngestionPipeline

# process nodes with metadata extractors
pipeline = IngestionPipeline(transformations=[node_parser, *extractors_1])

nodes_1 = pipeline.run(nodes=nodes, in_place=False, show_progress=True)

Parsing nodes: 0it [00:00, ?it/s]

0it [00:00, ?it/s]


In [ ]:
print(nodes_1[3].get_content(metadata_mode="all"))

IndexError: list index out of range

In [ ]:
# # process nodes with metadata extractor
# pipeline = IngestionPipeline(transformations=[node_parser, *extractors_2])

# nodes_2 = pipeline.run(nodes=nodes, in_place=False, show_progress=True)

# print(nodes_2[3].get_content(metadata_mode="all"))

# Chat Engine

In [ ]:
chat_engine = vector_index.as_chat_engine(chat_mode="condense_question")

In [ ]:
response = chat_engine.chat("Who is the principal of Kongu Engineering College?")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [ ]:
print(response)

Based on the context information provided, the principal of Kongu Engineering College is not explicitly mentioned. However, according to the conversation, the college is affiliated with Anna University Chennai, and the institution has a dedicated Trust called 'The Kongu Vellalar Institute of Technology Trust' that has been established to promote technical education in the region. The principal of the college is not specified in the provided context.


In [ ]:
response = chat_engine.chat("Can you explain it more?")

In [ ]:
print(response)

Based on the context information provided, Kongu Engineering College offers the following departments and programs:

Departments:

1. Department of Computer Science and Information Technology
2. Department of Electrical and Electronics Engineering
3. Department of Electronics and Communication Engineering
4. Department of Food Technology
5. Department of Chemical Engineering
6. Department of Mechanical Engineering
7. Department of Mechatronics Engineering
8. Department of Civil Engineering
9. Department of Artificial Intelligence (AI) and Data Science
10. Department of Computer Systems and Design
11. Department of Naturopathy

Programs:

1. Undergraduate programs: Bachelor of Science (B.Sc), Master of Science (M.Sc), Automobile Engineering
2. Postgraduate programs: Master of Engineering, Master of Business Administration (MBA), Master of Computer Applications (MCA)
3. Doctoral programs: Ph.D in Management, Ph.D in Computer Technology
4. Applied Sciences: Master of Science (M.Sc)
5. Doc

In [ ]:
chat_engine.reset()

In [ ]:
response = chat_engine.chat("Which University affiliates Kongu Engineering College?")

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


In [ ]:
print(response)

Great! Based on the context information provided, Kongu Engineering College is affiliated with Anna University Chennai.


# Prompt Engineering

In [ ]:
text_qa_template_str = (
    "You are a banking and finance expert and has access to all the earnings report.\n"
    "Always answer the query only using the provided context information, "
    "and not prior knowledge.\n"
    "Some rules to follow:\n"
    "1. Never directly reference the given context in your answer.\n"
    "2. Avoid statements like 'Based on the context, ...' or "
    "'The context information ...' or anything along "
    "those lines."
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Answer the question: {query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [ ]:
response = vector_index.as_query_engine(
    text_qa_template = text_qa_template).query("How does the Capital and Liquidity Position of Axis Bank looks like in the recent quarter?")

In [ ]:
display_response(response)

**`Final Response:`** Great! Based on the context information provided, Kongu Engineering College is affiliated with Anna University Chennai.

In [ ]:
pprint_response(response, show_source=True)

Final Response: Great! Based on the context information provided,
Kongu Engineering College is affiliated with Anna University Chennai.
______________________________________________________________________
Source Node 1/1
Node ID: 9059c625-1b61-481a-94e7-f4cb70a99d44
Similarity: 0.7654877078103297
Text: Kongu Engineering College establishes Center of Excellence (CoE)
to enhance the skills and knowledge of students and faculty in various
emerging technologies. The CoEs are equipped with state-of-the-art
infrastructure and facilities relevant to the specific field of study,
includes advanced laboratories, specialized equipments, software, and
oth...


In [ ]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='9059c625-1b61-481a-94e7-f4cb70a99d44', embedding=None, metadata={'file_name': 'KEC_02.docx', 'file_path': 'sample_data/pdfs/KEC_02.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 25263, 'creation_date': '2024-01-19', 'last_modified_date': '2024-01-19', 'last_accessed_date': '2024-01-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='d65736f9-0120-412f-85d2-24932bd1bde1', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_name': 'KEC_02.docx', 'file_path': 'sample_data/pdfs/KEC_02.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 25263, 'creation_date':

# End